#### Цели исследования:
- Анализ базы данных
- Решение поставленных задач

### Подключение к базе данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

### Исследование таблиц

In [2]:
query = "SELECT * FROM books LIMIT 3"
pd.io.sql.read_sql(query, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135


In [3]:
query = "SELECT * FROM ratings LIMIT 3"
pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5


In [4]:
query = "SELECT * FROM reviews LIMIT 3"
pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...


In [5]:
query = "SELECT * FROM publishers LIMIT 3"
pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books


In [6]:
query = "SELECT * FROM authors LIMIT 3"
pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie


### Выполнение заданий

#### 1. Посчитать сколько книг вышло после 1 января 2000 года

In [7]:
query1 = "SELECT COUNT(book_id) FROM books WHERE publication_date > '2000-01-01'"
pd.io.sql.read_sql(query1, con = engine) 

,count
0,819


После 1 января 2000 года вышло 819 книг.

#### 2. Посчитать количество обзоров и среднюю оценку для каждой книги

In [8]:
query2 = "SELECT b.title as book_title, COUNT(DISTINCT(rev.review_id)) as reviews_count, AVG(rat.rating) as avg_rating from books as b LEFT OUTER JOIN ratings AS rat ON b.book_id = rat.book_id LEFT OUTER JOIN reviews AS rev ON b.book_id = rev.book_id GROUP BY b.book_id"
pd.io.sql.read_sql(query2, con = engine)

,book_title,reviews_count,avg_rating
0,'Salem's Lot,2,3.666667
1,1 000 Places to See Before You Die,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,1491: New Revelations of the Americas Before C...,2,4.500000
4,1776,4,4.000000
...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),3,3.666667
996,Xenocide (Ender's Saga #3),3,3.400000
997,Year of Wonders,4,3.200000
998,You Suck (A Love Story #2),2,4.500000


На несколько книг отсутствуют обзоры, но у большинства книг есть хотя бы несколько обзоров и оценок.

#### 3. Определить издательство, которое выпустило наибольшее число книг (толще 50 страниц)

In [9]:
query3 = "SELECT p.publisher FROM (SELECT * FROM books WHERE num_pages > 50) as b INNER JOIN publishers AS p ON b.publisher_id = p.publisher_id GROUP BY p.publisher ORDER BY COUNT(b.book_id) DESC LIMIT 1"
pd.io.sql.read_sql(query3, con = engine)

,publisher
0,Penguin Books


Издательство, выпустившее наибольшее число книг - Penguin Books.

#### 4. Определить автора с самой высокой средней оценкой книг (учитывать только книги с 50 и более оценками)

In [10]:
query4 = "SELECT df.author, AVG(df.avg) FROM (SELECT b.title, a.author, COUNT(rat.rating_id), AVG(rat.rating) FROM books AS b INNER JOIN ratings AS rat ON b.book_id = rat.book_id INNER JOIN authors AS a ON b.author_id = a.author_id GROUP BY b.title, a.author HAVING COUNT(rat.rating_id) >= 50) as df GROUP BY df.author ORDER BY AVG(df.avg) DESC LIMIT 1"
pd.io.sql.read_sql(query4, con = engine)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


Автор с самой высокой средней оценкой книг - J.K. Rowling.

#### 5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [11]:
query5 = "SELECT ROUND(AVG(f.count)) as average FROM (SELECT username, COUNT(review_id) FROM reviews WHERE username IN (SELECT username FROM (SELECT username, COUNT(book_id) FROM ratings GROUP BY username) AS d WHERE d.count > 50) GROUP BY username) as f"
pd.io.sql.read_sql(query5, con = engine)

,average
0,24.0


Среднее количество обзоров от активных пользователей, поставивших больше 50 оценок - 24.